In [1]:
import numpy as np
import random
import math

In [2]:
def LineGenerate(lenght, m):
    """
    Creating a line with m random segments
    """
    line = np.zeros(lenght,)
    ab = np.array(sorted(random.sample(range(0, lenght-1), m*2))) # random start and end of the segments
    ab = ab.reshape((m,2)) # start and end of the segments coordinates
    # positive points coordinates
    ones = np.arange(ab[0,0],ab[0,1])
    for i in range(1,m):
        ones = np.concatenate((ones, np.arange(ab[i,0],ab[i,1])))
    line[ones] = 1  # put the generated segments on the line
    return line, ones, ab

In [3]:
def FindAssociated(c, ab, base, L):
    # distance from the farthest point to the base
    b = math.sqrt(c**2+base[1]**2)
    # distance from the farthest point to greedy
    dist = (L**2-2*L*b)/(2*(L-b-c))
    c_ass = c-dist
    for i in range(0,ab.shape[0]):
        if (c_ass > ab[i-1,1]) & (c_ass < ab[i,0]):
            c_ass = ab[i,0]
        if (c_ass < ab[0,0]):
            c_ass = ab[0,0]  
    return c_ass


In [4]:
def Candidates(base, L, ab):
    on_gap = False
    xB = ab[ab.shape[0]-1,1]
    c = np.array(xB)
    while not on_gap:
        # distance from the farthest point to the base
        b = math.sqrt(xB**2+base[1]**2)
        # distance from the farthest point to greedy
        dist = (L**2-2*L*b)/(2*(L-b-xB))
        # greedy point coordinate
        xA = xB-dist
        c = np.append(c, xA)
        xB = xA
        for i in range(0,ab.shape[0]):
            if (xA > ab[i-1,1]) & (xA < ab[i,0]):
                on_gap = True 
            if (xA < ab[0,0]):
                on_gap = True 
    c = c[:c.size-1]
    return c

In [5]:
def AllCandidates(base, L, ab):
    ab_cut = ab
    c = np.empty(0)
    while not (ab_cut.size == 0):
        c_i = Candidates(base, L, ab_cut)
        c = np.append(c, c_i)       
        ab_cut = ab_cut[:ab_cut.shape[0]-1]
    return c

In [6]:
def SegmentNumber(c, ab):
    for i in range(0,ab.shape[0]):
        if (c >= ab[i,0]) & (c <= ab[i,1]):
            return i  

In [7]:
def TourLenght(point1, point2, base):
    return max(point1, point2) - min(point1, point2) + math.sqrt(point1**2+base[1]**2) + math.sqrt(point2**2+base[1]**2)

In [8]:
def SeparateLeftRight(ab, base):
    ab_right = ab[np.argmax(ab[:,0] > base[0]):] - base[0]
    ab_left = np.flip(base[0] - ab[:np.argmax(ab[:,1] > base[0])])
    for i in range(0,ab.shape[0]):
        if (base[0] > ab[i,0]) & (base[0] < ab[i,1]):
            central_right = np.array([[base[0], ab[i,1]]])-base[0]
            central_left = -(np.array([[base[0],ab[i,0]]])-base[0])
            ab_right = np.append(central_right, ab_right, axis=0)
            ab_left = np.append(central_left, ab_left, axis=0)
    return ab_left, ab_right

In [9]:
def FindCentralTours(c_left, c_right , base, L):
    j = 0
    central_tours = np.empty((0,3))
    tour_lenght = 0
    while (tour_lenght <= L) and (j < c_left.size):
        central_tours_1 = np.empty((0,3))
        i = 0
        while (tour_lenght <= L) and (i < c_right.size):
            dist_left = math.sqrt(c_left[j]**2+base[1]**2)
            dist_right = math.sqrt(c_right[i]**2+base[1]**2)
            tour_lenght = c_left[j] + c_right[i] + dist_left + dist_right
            tour = np.array((c_left[j], c_right[i], tour_lenght)).reshape(1,3)
            central_tours_1 = np.append(central_tours_1, tour, axis=0)
            i += 1
        central_tours = np.append(central_tours, central_tours_1[central_tours_1[:,2]<=L,:], axis=0)
        tour_lenght = 0
        j += 1
    return central_tours

In [10]:
def DPwithPoints(c, ab, base, L):   
    c = np.sort(c)  
    E = np.zeros(c.size) # check if zeros are ok
    Start_Points = np.zeros((c.size,c.size)) # check if zeros are ok
    End_Points = np.zeros((c.size,c.size))

    for k in range(0, c.size):
        c_ass = FindAssociated(c[k], ab, base, L)
        
        j = np.arange(SegmentNumber(c_ass, ab), SegmentNumber(c[k], ab)+1)  
    
        if c_ass == ab[0,0]:
            E[k] = TourLenght(c[k], ab[0,0], base)  
            
            Start_Points[k,0] = c[k]
            End_Points[k,0] = ab[0,0]
            
        elif c_ass in ab[1:,0]:      
            
            func = np.empty(0)
            for i in j:
                func = np.append(func, TourLenght(c[k], ab[i,0], base) + E[c.tolist().index(ab[i-1,1])])
            E[k] = min(func)
            index_min = np.argmin(func)+j[0]
            
            Start_Points[k,:] = Start_Points[c.tolist().index(ab[index_min-1,1]),:]
            End_Points[k,:] = End_Points[c.tolist().index(ab[index_min-1,1]),:]
                                   
            Start_Points[k, np.argmax(Start_Points[k,:]==0)] = c[k]
            End_Points[k, np.argmax(End_Points[k,:]==0)] = ab[index_min,0]
    
        else:             
            func = np.empty(0)
            if (c_ass < ab[SegmentNumber(c[k], ab),0]): 
                for i in j[1:]:
                    func = np.append(func, TourLenght(c[k], ab[i,0], base) + E[c.tolist().index(ab[i-1,1])])  
            else:
                func = [1000000.0]
            func2 = L + E[c.tolist().index(c_ass)]
            
            if (func2>min(func)):
                index_min = np.argmin(func)+j[0]
                Start_Points[k,:] = Start_Points[c.tolist().index(ab[index_min-1,1]),:]
                End_Points[k,:] = End_Points[c.tolist().index(ab[index_min-1,1]),:]
                End_Points[k,np.argmax(End_Points[k,:]==0)] = ab[index_min,0]
            else:
                Start_Points[k,:] = Start_Points[c.tolist().index(c_ass),:] 
                End_Points[k,np.argmax(End_Points[k,:]==0)] = c_ass
                
            Start_Points[k,np.argmax(Start_Points[k,:]==0)] = c[k]
                                 
            E[k] = min(func2, min(func))
            
    Start_Points = Start_Points[c.size-1]
    End_Points = End_Points[c.size-1]
    return E[c.size-1], Start_Points[Start_Points>0], End_Points[End_Points>0]

In [11]:
def OneSideWithPoints(c, c_first, ab):
    E = np.zeros(c_first.size)
    StartPoints = np.zeros((c_first.size,c.size))
    EndPoints = np.zeros((c_first.size,c.size))
    ab_i = ab.astype(np.float64)
    for i in range(0, c_first.size):
        if c_first[i] != ab_i[0,1]:
            ab_i[0,0] = c_first[i]
        else:
            ab_i = ab[ab[:,0]>c_first[i]].astype(np.float64)
        if (c[c>c_first[i]].size != 0):
            E[i],  Start_Points, End_Points = DPwithPoints(c[c>c_first[i]], ab_i, base, L) 
            StartPoints[i,:Start_Points.size] = Start_Points
            EndPoints[i,:End_Points.size] = End_Points
            
        else: E[i] = 0
    return E, StartPoints, EndPoints

In [12]:
def JoinTours(StartPoints, EndPoints):
    Start = np.array([StartPoints[StartPoints>0]])
    End = np.array([EndPoints[EndPoints>0]])
    
    return np.append(Start, End, axis = 0)

In [13]:
def BothSidesWithPoints(ab, base, L):
    ab_left, ab_right = SeparateLeftRight(ab, base)
    
    c_right = np.sort(AllCandidates(base, L, ab_right))
    c_left =  np.sort(AllCandidates(base, L, ab_left))
    
    central_tours = FindCentralTours(c_left, c_right , base, L)
    
    c_left_first = np.unique(np.sort(central_tours[:,0]))
    c_right_first = np.unique(np.sort(central_tours[:,1]))
    
    E_left, StartPoints_left, EndPoints_left = OneSideWithPoints(c_left, c_left_first, ab_left)
    E_right, StartPoints_right, EndPoints_right = OneSideWithPoints(c_right, c_right_first, ab_right)
        
    left_central = np.append(c_left_first.reshape(E_left.shape[0],1), E_left.reshape(E_left.shape[0],1), axis=1)
    right_central = np.append(c_right_first.reshape(E_right.shape[0],1), E_right.reshape(E_right.shape[0],1), axis=1)
      
    totalL = np.zeros(central_tours.shape[0])
    for i in range (0, central_tours.shape[0]):
        a = tuple(np.argwhere(left_central == central_tours[i,0])[0])
        b = tuple(np.argwhere(right_central == central_tours[i,1])[0])
        totalL[i] = left_central[a[0],1] + right_central[b[0],1] + float(central_tours[i,2])
        
    c = tuple(np.argwhere(left_central == central_tours[np.argmin(totalL),0])[0])
    d = tuple(np.argwhere(right_central == central_tours[np.argmin(totalL),1])[0])
              

    E_left_max, StartPoints_left_max, EndPoints_left_max = DPwithPoints(c_left, ab_left, base, L)
    E_right_max, StartPoints_right_max, EndPoints_right_max = DPwithPoints(c_right, ab_right, base, L)
    
    if (E_left_max + E_right_max) < min(totalL):
        Tour_right = JoinTours(StartPoints_right_max, EndPoints_right_max) + base[0]
        Tour_right = Tour_right[ ::-1, :]
        Tour_left = base[0] - JoinTours(StartPoints_left_max, EndPoints_left_max)
        Tour = np.append(Tour_left, Tour_right, axis = 1)
        
    else:       
        Tour_right = JoinTours(StartPoints_right[d[0]], EndPoints_right[d[0]]) + base[0]
        Tour_right = Tour_right[ ::-1, :]
        Tour_left = base[0] - JoinTours(StartPoints_left[c[0]], EndPoints_left[c[0]])
        Tour_central = np.array([[base[0] - central_tours[np.argmin(totalL),0], central_tours[np.argmin(totalL),1] + base[0]]]).reshape((2, 1))
        
        Tour = np.append(Tour_left, Tour_central, axis = 1)
        Tour = np.append(Tour, Tour_right, axis = 1)
        
    TotalLenght = min((E_left_max + E_right_max), min(totalL)) 
    
    for i in range(Tour.shape[1]):
        print('Tour', i+1, 'start:', Tour[0,i])
        print('Tour', i+1, 'end:', Tour[1,i])
        print('Tour', i+1, 'lenght:', TourLenght(abs(Tour[0,i]-base[0]), abs(Tour[1,i]-base[0]), base))
    print('Total number of tours:', Tour.shape[1])
    print('Total lenght:', TotalLenght)
    
    return Tour, TotalLenght

In [14]:
line, ones, ab = LineGenerate(400, 15)
base = np.array((200,500)) # base station coordinates
L = 1120

In [15]:
ab

array([[  8,  13],
       [ 22,  32],
       [ 36,  47],
       [ 81,  82],
       [ 83,  95],
       [110, 135],
       [142, 146],
       [162, 223],
       [226, 228],
       [260, 268],
       [294, 297],
       [310, 318],
       [332, 356],
       [371, 373],
       [383, 390]])

In [16]:
Tour, TotalLenght = BothSidesWithPoints(ab, base, L)
Tour = np.transpose(Tour)
Tour = Tour - base[0]

Tour 1 start: 81.0
Tour 1 end: 146.0
Tour 1 lenght: 1081.8734983549527
Tour 2 start: 8.0
Tour 2 end: 47.0
Tour 2 lenght: 1097.4821279830378
Tour 3 start: 162.0
Tour 3 end: 228.0
Tour 3 lenght: 1012.225307170117
Tour 4 start: 260.0
Tour 4 end: 318.0
Tour 4 lenght: 1075.322472904019
Tour 5 start: 332.0
Tour 5 end: 390.0
Tour 5 lenght: 1110.0137092596042
Total number of tours: 5
Total lenght: 5432.91711567173


In [17]:
def FindTheFarthest(base,first,last):
    """
    Looking for the farthest point from the base
    """
    # distances to the base station
    fdist = math.sqrt(first**2+base[1]**2)
    ldist = math.sqrt(last**2+base[1]**2)

    # comparing the distances
    if fdist > ldist:
        farthest = first
        dist = fdist
    else:
        farthest = last
        dist = ldist
    
    return farthest, dist

In [47]:
Tour = np.sort(Tour, axis = 0)
Tour

array([[-192., -153.],
       [-119.,  -54.],
       [ -38.,   28.],
       [  60.,  118.],
       [ 132.,  190.]])

In [19]:
def TourLength(tour):
    dist_left = math.sqrt(tour[0]**2+base[1]**2)
    dist_right = math.sqrt(tour[1]**2+base[1]**2)
    tour_lenght = abs(tour[0]-tour[1]) + dist_left + dist_right
    return tour_lenght

In [20]:
def ToTheMinDisribution(Tour):
    Tour1 = np.empty(0)
    Tour2 = np.empty(0)
    M1 = 0
    M2 = 0
    while Tour.size>1:
        farthest, dist = FindTheFarthest(base,Tour[0,0],Tour[Tour.shape[0]-1,1])
        if farthest == Tour[0,0]:
            if M1 < M2:
                Tour1 = np.append(Tour1, Tour[0])
                M1 += TourLength(Tour[0])
            else:
                Tour2 = np.append(Tour2, Tour[0])
                M2 += TourLength(Tour[0])
            Tour = Tour[1:]
        else:
            if M1 < M2:
                Tour1 = np.append(Tour1, Tour[Tour.shape[0]-1])
                M1 += TourLength(Tour[Tour.shape[0]-1])
            else:
                Tour2 = np.append(Tour2, Tour[Tour.shape[0]-1])
                M2 += TourLength(Tour[Tour.shape[0]-1])
            Tour = Tour[:Tour.shape[0]-1]
    Tour1 = Tour1.reshape(Tour1.shape[0]//2,2)
    Tour2 = Tour2.reshape(Tour2.shape[0]//2,2)
    if M1 > M2:
        Tour1, Tour2 = Tour2, Tour1
        M1, M2 = M2, M1

    print("T1 =", Tour1)
    print("T1 total length: M1 =", M1)
    print("T2 =",Tour2)
    print("T2 total length: M2 =", M2)   
    return Tour1, Tour2, M1, M2

In [21]:
def OneToEachDisribution(Tour):
    Tour1 = np.empty(0)
    Tour2 = np.empty(0)
    while Tour.size>1:
        farthest, dist = FindTheFarthest(base,Tour[0,0],Tour[Tour.shape[0]-1,1])
        if farthest == Tour[0,0]:
            Tour1 = np.append(Tour1, Tour[0])
            Tour = Tour[1:]
        else:
            Tour1 = np.append(Tour1, Tour[Tour.shape[0]-1])
            Tour = Tour[:Tour.shape[0]-1]
        
        farthest, dist = FindTheFarthest(base,Tour[0,0],Tour[Tour.shape[0]-1,1])
        if farthest == Tour[0,0]:
            Tour2 = np.append(Tour2, Tour[0])
            Tour = Tour[1:]
        else:
            Tour2 = np.append(Tour2, Tour[Tour.shape[0]-1])
            Tour = Tour[:Tour.shape[0]-1]  
            
    return Tour1, Tour2

In [185]:
Tour1, Tour2, M1, M2 = ToTheMinDisribution(Tour)

T1 = [[132. 190.]
 [ 60. 118.]]
T1 total length: M1 = 2185.336182163623
T2 = [[-192. -153.]
 [-119.  -54.]
 [ -38.   28.]]
T2 total length: M2 = 3247.5809335081076


In [186]:
"STEP 4"
l = M2 - M1
A = M1 + l/2

In [187]:
"STEP 5"
SubTour2 = np.empty(0)
for tour in Tour2:
    dist_left = math.sqrt(tour[0]**2+base[1]**2)
    dist_right = math.sqrt(tour[1]**2+base[1]**2)
    if (dist_left + dist_right)<l:
        SubTour2 = np.append(SubTour2, tour)
SubTour2 = SubTour2.reshape(SubTour2.shape[0]//2,2)

In [188]:
L

1120

In [189]:
"STEP 6"
SubTour11 = np.empty(0)
for tour in Tour1:
    if TourLength(tour)<L:
        SubTour11 = np.append(SubTour11, tour)
SubTour11 = SubTour11.reshape(SubTour11.shape[0]//2,2)

In [190]:
SubTour1 = np.empty(0)
for tour in SubTour11:
    a = np.where(Tour == tour)[0][0]
    if a < Tour.shape[0]-1:
        if (TourLength(Tour[a+1]) == L) and (Tour[a,1] == Tour[a+1,0]):
            SubTour1 = np.append(SubTour1, tour)   
SubTour1 = SubTour1.reshape(SubTour1.shape[0]//2,2)       
print(SubTour1)

[]


In [191]:
"STEP 7"
xr = np.empty(0)
r = np.empty(0)
for tour in SubTour2:
    if abs(tour[1]) > abs(tour[0]):
        closest = abs(tour[0])
    else:
        closest = abs(tour[1])
        
    a = math.sqrt(closest**2+base[1]**2)
    xr = np.append (xr, (l-a*2)/2)
    d = math.sqrt((closest+x)**2+base[1]**2)
    r = np.append(r, d - dist_right - x)   
    print("t = ", tour)      
    print("xr = ", xr)    
    print("d = ", d)
    print("r = ", r)
    print("_____________")
# What about the middle segment?

t =  [-192. -153.]
xr =  [8.23711101]
d =  532.5534574311515
r =  [1.43108176]
_____________
t =  [-119.  -54.]
xr =  [ 8.23711101 28.2148295 ]
d =  507.06317666276726
r =  [  1.43108176 -24.05919901]
_____________
t =  [-38.  28.]
xr =  [ 8.23711101 28.2148295  30.33898937]
d =  503.39193247437595
r =  [  1.43108176 -24.05919901 -27.7304432 ]
_____________


In [192]:
"STEP 8"
xs = np.empty(0)
s = np.empty(0)
for tour in SubTour1:
    if abs(tour[1]) > abs(tour[0]):
        farthest = abs(tour[1])
    else:
        farthest = abs(tour[1])  
        
    xs = np.append (xs,  l/2)
    c = math.sqrt(farthest**2+base[1]**2)
    d = math.sqrt((farthest+x)**2+base[1]**2)
    s = np.append(s, d - c - x)
    print("t = ", tour)      
    print("x = ", x)
    print("c = ", c)    
    print("d = ", d)
    print("r = ", s)
    print("_____________")

In [193]:
"STEP 9"
# What is s?
# Minimum negative or abs?
if r.size > 0:
    minr = np.min(r)
    if s.size == 0:
        minrs = minr
        mins = 0
if s.size > 0:
    mins = np.min(s)
    if r.size == 0:
        minrs = mins
        minr = 0
if (s.size > 0) and (s.size > 0):
    minrs = np.min(minr, mins)

In [194]:
"STEP 10"
# We are going from the base one by one, right?
# Do we need to divide them all?
if minrs.size > 0:
    if minrs == minr:
        a = Tour2[Tour2.shape[0]-1, 0]
        Tour2[Tour2.shape[0]-1, 0] = Tour2[Tour2.shape[0]-1, 0] + xr[Tour2.shape[0]-1] 

newTour = np.array([a, Tour2[Tour2.shape[0]-1, 0]])
Tour1 = np.append(Tour1, newTour)
Tour1 = Tour1.reshape(Tour1.shape[0]//2,2)

In [195]:
xr

array([ 8.23711101, 28.2148295 , 30.33898937])

In [202]:
xr[Tour2.shape[0]-1] 

30.338989366346482

In [197]:
Tour1

array([[132.        , 190.        ],
       [ 60.        , 118.        ],
       [-38.        ,  -7.66101063]])

In [198]:
Tour2

array([[-192.        , -153.        ],
       [-119.        ,  -54.        ],
       [  -7.66101063,   28.        ]])

In [199]:
Tour1Lenght = 0
Tour2Lenght = 0
for tour in Tour1:
    Tour1Lenght += TourLength(tour)
for tour in Tour2:
    Tour2Lenght += TourLength(tour)    

In [200]:
Tour1Lenght

3217.175780033881

In [201]:
Tour2Lenght

3215.8587109172295

In [ ]:
# TODO: the total distances are not equal. Check x calculation. 
# Improve the STEP 10
# STEP 11
# Questions
# Improve the code and maybe the algrithm structure